Finnish municipalities with health district information is an Excel spreadsheet from Kuntaliitto: https://www.kuntaliitto.fi/sosiaali-ja-terveysasiat/sairaanhoitopiirien-jasenkunnat File Shp_jäsenkunnat_2020.xls, sheet kunnat_shp_2020_ aakkosjärj.

"shp" stands for "sairaanhoitopiiri" (health district in Finnish). I have changed the name of the file to Shp_jasenkunnat_2020.xls and sheet to kunnat_shp_2020_ aakkosjarj

Municipality polygons from National Land Survey of Finland web feature service:
https://pta.spatineo-devops.com/sofp/collections/su_kansallinen_meri_1000k_wgs84/2020/items?tessellation=kunta&limit=1000&f=html&f=json
info: https://pta.spatineo-devops.com/sofp/collections/su_kansallinen_meri_1000k_wgs84/2020/items?&tessellation=kunta&limit=1000

Population count for each municipality from Statistics Finland: https://www.stat.fi/org/avoindata/paikkatietoaineistot/vaesto_tilastointialueittain.html

Average income from the Paavo (zip-code) database: https://www.stat.fi/org/avoindata/paikkatietoaineistot.html


In [ ]:
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xlrd   #to be able to read Excel


In [ ]:
#1. Health district data

df_orig = pd.read_excel("Shp_jasenkunnat_2020.xls", sheet_name="kunnat_shp_2020_ aakkosjarj",
                     header=3)
df_orig.dropna(inplace=True)
df_orig.head()

In [ ]:
healthDistrict = df_orig.copy()
print(healthDistrict.shape)
healthDistrict.tail()

In [ ]:
healthDistrict.rename(columns={"kunta-\nkoodi":"code", 'sairaanhoitopiiri':'healthCareDistrict'},
          inplace=True)
healthDistrict = healthDistrict[['code','healthCareDistrict']]
healthDistrict.info()


In [ ]:
# Truncate and convert to character string
healthDistrict["code"] = healthDistrict["code"].astype(int)
healthDistrict["code"] = healthDistrict["code"].astype('str')
healthDistrict.head()


In [ ]:
# Add missing zeros to municipality codes
healthDistrict["code"] = healthDistrict["code"].apply(lambda x: "00" + x if len(x)==1 else x)
healthDistrict["code"] = healthDistrict["code"].apply(lambda x: "0" + x if len(x)==2 else x)
healthDistrict.head()


In [ ]:
healthDistrict.info()

In [ ]:
#2. GIS layer data

# For available features, see http://geo.stat.fi/geoserver/tilastointialueet/wfs?request=GetCapabilities
url = "https://pta.spatineo-devops.com/sofp/collections/su_kansallinen_meri_1000k_wgs84/2020/items?tessellation=kunta&limit=1000&f=html&f=json"
geodata_orig = gpd.read_file(url)

In [ ]:
# There are 310 municipalities in Finland in 2020
geodata = geodata_orig.copy()
print(geodata.shape)
geodata.head()

In [ ]:
geodata = geodata[['code', 'geometry']]
geodata.tail()

In [ ]:
geodata.info()

In [ ]:
#Plot of municipalities
geodata.plot()

Population count for each municipality from Statistics Finland: https://www.stat.fi/org/avoindata/paikkatietoaineistot/vaesto_tilastointialueittain.html

WFS: http://geo.stat.fi/geoserver/vaestoalue/wfs 
Note: Valtimo merged with Nurmes in 2020. Belongs to Pohjois-Karjala health care district. 

In [ ]:
# For available features, see http://geo.stat.fi/geoserver/vaestoalue/
#wfs?request=GetCapabilities

url = "http://geo.stat.fi/geoserver/vaestoalue/wfs?request=GetFeature&typename=vaestoalue:kunta_vaki2018&outputformat=JSON"
pop_orig = gpd.read_file(url)

In [ ]:
pop = pop_orig.copy()
print(pop.shape)
print(list(pop))
pop.head()

In [ ]:
#Select and rename columns
pop = pop[["kunta", "name", "vaesto","ika_65_"]]
pop.rename(columns={'kunta':'code', 'vaesto':'population', 'ika_65_':'age_65'}, inplace=True)
pop.tail()

In [ ]:
# Check length, in 2020, there are 310 Municipalities. 
# 2019 data still contains Valtimo which was merged with Nurmes at the end of 2019
pop.loc[pop['name'] == 'Valtimo']

In [ ]:
pop.loc[pop['name'] == 'Nurmes']

In [ ]:
pop.loc[292, 'name'] = 'Nurmes'
pop.loc[292, 'code'] = 541

temp = pop.loc[pop['name'] == 'Nurmes']
temp

In [ ]:
pop.loc[176, 'population'] = sum(temp.population) 
pop.loc[176, 'age_65'] = sum(temp.age_65) 
pop = pop.drop(292)   #drop Valtimo
print(pop.shape)
pop.loc[176]

In [ ]:
pop.info()

In [ ]:
geodata = geodata.merge(pop[["code", "name", "population", "age_65"]], on="code")
geodata.head()

In [ ]:
#Join Health district to geodata
geodata = geodata.merge(healthDistrict, on="code", how="left")
geodata.tail(8)

In [ ]:
# Municipalities in the Åland island did not have a matching health care district in the data
# count the number of NaN values in each column
print(geodata.isnull().sum())
geodata[geodata.healthCareDistrict.isnull()].name

In [ ]:
# Update "Ahvenanmaa" as the health care district for Åland municipalities (16 municipalities in total)
geodata.loc[geodata.healthCareDistrict.isnull(),'healthCareDistrict'] = "Ahvenanmaa"
geodata.healthCareDistrict.value_counts()

In [ ]:
geodata.info()

In [ ]:
#Create polygons for health care districts
# Dissolve (=combine) municipality polygon geometries for each health care district
# In the geopandas library, we can aggregate geometric features using the dissolve function.

districts = geodata.dissolve(by='healthCareDistrict', aggfunc="sum")
districts.reset_index(inplace=True)
districts.head(3)

In [ ]:
#calclulate percentage old
districts['perc_pop_over_65'] = round( (districts['age_65']/districts['population']*100) , 1)
#districts.to_file("HealthDistricts.geojson", driver="GeoJSON")  #can be exported as a geojson file
districts

In [ ]:
import plotly.graph_objs as go
import plotly.express as px

df_districts = districts[['healthCareDistrict', 'population', 'perc_pop_over_65']]

fig = px.choropleth_mapbox(df_districts,
                           geojson=districts,   
                           locations="population",
                           featureidkey="properties.population",
                           color="perc_pop_over_65",
                           color_continuous_scale="Viridis",
                           range_color=(18, 32),
                           mapbox_style="carto-positron",
                           zoom=3, center={"lat": 65, "lon": 26},
                           opacity=0.7,
                           labels={'perc_pop_over_65':'percentage of population over 65'},
                           hover_name="healthCareDistrict"
                           )

fig.show()

In [ ]:
#3. Calculate average income

#Paavo: https://www.stat.fi/org/avoindata/paikkatietoaineistot.html, zip-code info for Finland
url = "http://geo.stat.fi/geoserver/wfs?service=WFS&version=1.0.0&request=GetFeature&typeName=postialue:pno_tilasto&outputFormat=json"
zip_code_orig = gpd.read_file(url)
zip_code_orig.tail()

In [ ]:
df = zip_code_orig.copy()
print(df.shape)
print(list(df))

In [ ]:
#'hr_ktu' is average yearly income(€) for inhabitants over 18 years in 2017.   
#'ko_ika18y' is the number of inhabitants of age 18 or over (2018)

df = df[['postinumeroalue', 'kunta', 'ko_ika18y', 'hr_ktu']]

#kunta is municipality, replaced with code
df.rename(columns={'postinumeroalue': 'zip_code', 'kunta': 'code', 'ko_ika18y':'age18', 
                   'hr_ktu':'average_income'}, inplace = True)

df['total_av_income'] = df['average_income']* df['age18'] 
df['total_av_income'] = df['total_av_income'].astype(int)
df.head()

In [ ]:
df.info()

In [ ]:
# count the number of NaN values in each column
print(df.isnull().sum())

In [ ]:
#Assuming if avaraeg income less than 1 €/ year, data is missing and rows are dropped.
temp = df.loc[df['average_income'] <= 1]
temp

In [ ]:
#drop temp index number
drop_list = temp.index.to_list()
df = df.drop(drop_list)
print(df.shape)
df.tail(10)

In [ ]:
df = df [['code', 'age18', 'total_av_income']]
df = df.groupby(['code']).sum()
df.reset_index(inplace=True)  
print(df.shape)
df.head()

In [ ]:
#292= Valtimo is correctly missing
print(df['code'].to_list())

In [ ]:
#Join Total Average Income to geodata
geo_df = geodata.merge(df, on="code", how="left")
print(list(geo_df))
geo_df.tail(8)

In [ ]:
districts_income = geo_df.dissolve(by='healthCareDistrict', aggfunc="sum")
districts_income['income_pro_person'] = round( districts_income['total_av_income']/districts_income['age18'] , 0)
districts_income['income_pro_person'] = districts_income['income_pro_person'].astype(int)
districts_income.reset_index(inplace=True)
districts_income

In [ ]:
df_income = districts_income[['healthCareDistrict', 'population', 'income_pro_person']]
#df_income.to_csv('df_income.csv')
df_income.head(2)

In [ ]:
fig = px.choropleth_mapbox(df_income,
                           geojson= districts_income,  
                           locations="population",
                           featureidkey="properties.population",
                           color="income_pro_person",
                           color_continuous_scale="Viridis",
                           range_color=(20000, 30000),
                           mapbox_style="carto-positron",
                           zoom=3, center={"lat": 65, "lon": 26},
                           opacity=0.7,
                           labels={'income_pro_person':'Income pro person (over 18 y)'},
                           hover_name="healthCareDistrict"
                           )

fig.show()